In [69]:
import sys
!{sys.executable} -m pip install rdflib yatter morph-kgc

In [2]:
from os import getcwd
from os.path import join, dirname
folder = dirname(getcwd())

In [46]:
from rdflib import Graph
# Load RDF data from file
g = Graph()
rdf_file = join(folder, "rdf/digital-competence-framework-skos-ap-eu.rdf")  # Replace with the actual path to your RDF file
g.parse(rdf_file, format="xml")     # Specify the appropriate format if not XML

# SPARQL query
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    
    SELECT DISTINCT ?label ?id
    WHERE {
      ?s skos:prefLabel ?label ;
          dc:identifier ?id .
      FILTER (!CONTAINS(?label, 'Level'))
    }
    ORDER BY ?label
"""

# Execute the SPARQL query
results = g.query(sparql_query)

# Print the query results
for row in results:
    label = str(row.label)[:3].strip()
    if len(label) == 2:
        label = label[:1]
    print(f"{label},{row.id}")

1,8b23nt41l4
1.1,9i6ta81748
1.2,3h9eab1489
1.3,4s23kdq383
2,2k3209g1wc
2.1,8ys96p08p4
2.2,9a5d39t62b
2.3,1dz4n8r554
2.4,y2s2861s9p
2.5,99vvo91542
2.6,7d4t7002v7
3,4b465z8p2e
3.1,u69o196gu6
3.2,sq0c9q6116
3.3,2mi55kl065
3.4,0690r7o8r2
4,1fz48677f4
4.1,aa985fr035
4.2,1b043jw49b
4.3,mg54my1580
4.4,o9e8s72z21
5,86xdxt8620
5.1,13z458t54u
5.2,00b51s9rc5
5.3,3bgx2a6278
5.4,860966ekgo


In [62]:
import yatter
from ruamel.yaml import YAML

# YARRRML to RML
yaml = YAML(typ='safe', pure=True)
mappings_file = join(folder, "mappings/corso-kg-yarrrml.yml")
rml_content = yatter.translate(yaml.load(open(mappings_file)))

# Print generated RML triples
print(rml_content)

# Save RML to file
rml_file = join(folder, "mappings/corso-kg-rml.ttl")
with open(rml_file, "w") as rml_output:
    rml_output.write(rml_content)

2023-07-18 14:32:50,435 | INFO: Translating YARRRML mapping to [R2]RML
2023-07-18 14:32:50,436 | INFO: RML content is created!
2023-07-18 14:32:50,446 | INFO: Mapping has been syntactically validated.
2023-07-18 14:32:50,447 | INFO: Translation has finished successfully.


@prefix kg: <http://www.cefriel.com/corso-kg/data/>.
@prefix ex: <http://www.cefriel.com/corso-kg/schema/>.
@prefix skos: <http://www.w3.org/2004/02/skos/core#>.
@prefix rr: <http://www.w3.org/ns/r2rml#>.
@prefix rml: <http://semweb.mmlab.be/ns/rml#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix ql: <http://semweb.mmlab.be/ns/ql#>.
@prefix d2rq: <http://www.wiwiss.fu-berlin.de/suhl/bizer/D2RQ/0.1#>.
@prefix foaf: <http://xmlns.com/foaf/0.1/>.
@prefix schema: <http://schema.org/>.
@prefix formats: <http://www.w3.org/ns/formats/>.
@prefix comp: <http://semweb.mmlab.be/ns/rml-compression#>.
@prefix void: <http://rdfs.org/ns/void#>.
@prefix fnml: <http://semweb.mmlab.be/ns/fnml#>.
@prefix grel: <http://users.ugent.be/~bjdmeest/function/grel.ttl#>.
@base <http://example.com/ns#>.


<unit_0> a rr:TriplesMap;

	rml:logicalSource [
		a rml:LogicalSource;
		rml:source 

In [75]:
import morph_kgc

config = """      
            [Competences]
            mappings: ./../mappings/corso-kg-rml.ttl
         """
# Generate the triples and load them to an RDFLib graph
g_rdflib = morph_kgc.materialize(config)
# Save to file
output_file = join(folder, "rdf/knowledge-graph.ttl")
g_rdflib.serialize(destination=output_file, format='ttl', encoding="utf-8")

2023-07-18 15:31:50,649 | DEBUG: CONFIGURATION: {'output_file': 'knowledge-graph', 'na_values': ',nan', 'safe_percent_encoding': '', 'read_parsed_mappings_path': '', 'write_parsed_mappings_path': '', 'mapping_partitioning': 'PARTIAL-AGGREGATIONS', 'logging_file': '', 'oracle_client_lib_dir': '', 'oracle_client_config_dir': '', 'udfs': '', 'output_dir': '', 'output_format': 'N-TRIPLES', 'only_printable_chars': 'no', 'infer_sql_datatypes': 'no', 'logging_level': 'INFO', 'number_of_processes': '16'}
2023-07-18 15:31:50,650 | DEBUG: DATA SOURCE `Competences`: {'mappings': './../mappings/corso-kg-rml.ttl'}
2023-07-18 15:31:51,456 | DEBUG: Removed self-join from mapping rule `#TM5`.
2023-07-18 15:31:51,461 | INFO: 11 mapping rules retrieved.
2023-07-18 15:31:51,478 | INFO: Mapping partition with 1 groups generated.
2023-07-18 15:31:51,480 | INFO: Maximum number of rules within mapping group: 11.
2023-07-18 15:31:51,482 | INFO: Mappings processed in 0.826 seconds.
2023-07-18 15:31:51,486 | DE

<Graph identifier=N9ac989585ce448a19865eda4d1f5c239 (<class 'rdflib.graph.Graph'>)>

In [76]:
q_res = g_rdflib.query('SELECT (COUNT(*) AS ?num_triples) WHERE { ?s ?p ?o }')

# Print the query results
for row in q_res:
    print(f"Number of triples generated: {row.num_triples}")

Number of triples generated: 1210


In [77]:
graph = Graph()
graph.parse(output_file, format='ttl')

<Graph identifier=N24f09c7bf0d74c5c8995da9b73afb130 (<class 'rdflib.graph.Graph'>)>